In [84]:
import numpy as np
import pandas as pd
import re
from tensorlayer.models.seq2seq import Seq2seq
from tensorlayer.models.seq2seq_with_attention import Seq2seqLuongAttention
from keras.models import Model
from keras.layers import Input, LSTM, Dense
data = pd.read_csv('chat.csv')

ModuleNotFoundError: No module named 'tensorlayer'

In [45]:
data.head()

,유저,챗봇,embedding,distance
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401...",0.361785
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949...",0.440894
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792...",0.255487
3,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045...",0.388310
4,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624...",0.331457


In [46]:
in_texts, out_texts = [], []
in_vocab = set()
out_vocab = set()
start_token = '\t'
end_token = '\n'

max_train_size = max(25000, len(data)-1)

In [69]:
for in_text, out_text in zip(data.유저, data.챗봇):
    out_text = start_token + out_text + end_token
    in_texts.append(in_text)
    out_texts.append(out_text)
    for char in in_text:
        if char not in in_vocab:
            in_vocab.add(char)
    for char in out_text:
        if char not in out_vocab:
            out_vocab.add(char)

In [70]:
in_vocab = sorted(in_vocab)
out_vocab = sorted(out_vocab)

in_vocab_size = len(in_vocab)
out_vocab_size = len(out_vocab)

In [71]:
max_encoder_seq_length = max([len(txt) for txt in in_texts])
max_decoder_seq_length = max([len(txt) for txt in out_texts])

In [72]:
in_token_index = dict([(char, i) for i, char in enumerate(in_vocab)])
out_token_index = dict([(char, i) for i, char in enumerate(out_vocab)])

In [73]:
reverse_in__token_char_index = dict((i, char) for char, i in in_token_index.items())
reverse_out_token_char_index = dict((i, char) for char, i in out_token_index.items())

In [74]:
encoder_in_data = np.zeros((len(in_texts), max_encoder_seq_length, in_vocab_size), dtype='float32')

decoder_in_data = np.zeros((len(out_texts), max_decoder_seq_length, out_vocab_size), dtype='float32')

In [75]:
decoder_out_data = np.zeros((len(in_texts), max_decoder_seq_length, out_vocab_size), dtype='float32')

In [76]:
for i, (in_text, out_text) in enumerate(zip(in_texts, out_texts)):
    for t, char in enumerate(in_text):
        encoder_in_data[i, t, in_token_index[char]] = 1.
    for t, char in enumerate(out_text):
        decoder_in_data[i, t, out_token_index[char]] = 1.
        if t > 0:
            decoder_out_data[i, t - 1, out_token_index[char]] = 1.

In [77]:
batch_size = 64
epochs = 100
num_units = 256

In [78]:
encoder_inputs = Input(shape=(None, in_vocab_size))
encoder = LSTM(num_units, return_state=True)

In [79]:
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [80]:
decoder_inputs = Input(shape=(None, out_vocab_size))
decoder_lstm = LSTM(num_units, return_sequences=True,return_state=True)

In [81]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

In [82]:
decoder_dense = Dense(out_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [83]:
model_one = Seq2seq(
    decoder_seq_length = decoder_seq_length,
    cell_enc=tf.keras.layers.GRUCell,
    cell_dec=tf.keras.layers.GRUCell,
    n_layer=3,
    n_units=256,
    embedding_layer=tl.layers.Embedding(vocabulary_size=vocabulary_size, embedding_size=emb_dim),
    )
    # Uncomment below statements if you have already saved the model
    # load_weights = tl.files.load_npz(name='model.npz')
    # tl.files.assign_weights(load_weights, model_)

NameError: name 'Seq2seq' is not defined

In [61]:
optimizer='rmsprop'
loss='categorical_crossentropy'
metrics=['acc']

In [62]:
model_one = Model([encoder_inputs, decoder_inputs],decoder_outputs )
model_one.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [63]:
model_one.fit(
    [encoder_in_data, decoder_in_data],
    decoder_out_data,
#     batch_size=batch_size,
    epochs= 2,
    validation_split=0.2
)

Epoch 1/2
 44/322 [===>..........................] - ETA: 2:41 - loss: 1.1043 - acc: 0.0380

KeyboardInterrupt: 

In [64]:
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [
    Input(shape=(num_units,)),
    Input(shape=(num_units,))
]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs,
    initial_state=thought_input
)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

In [65]:
decoder_model = Model(
    inputs=[decoder_inputs] + thought_input,
    outputs=[decoder_outputs] + decoder_states
)

In [66]:
def decode_seq(input_seq):
    thought = encoder_model.predict(input_seq)
    
    output_seq = np.zeros((1,1, out_vocab_size))
    output_seq[0,0, out_token_index[end_token]] = 1.
    stop_condition = False
    generated_seq = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([output_seq] + thought)
        
        generated_token_idx = np.argmax(output_tokens[0, -1, :])
        generated_char = reverse_out_token_char_index[generated_token_idx]
        generated_seq += generated_char
        if (generated_char == end_token or
               len(generated_seq) > max_decoder_seq_length
           ):
            stop_condition = True
            
        target_seq = np.zeros((1, 1, out_vocab_size))
        target_seq[0, 0, generated_token_idx] = 1.
        thought = [h, c]
    return generated_seq

In [67]:
def response(input_text):
    input_seq = np.zeros((1, max_encoder_seq_length, in_vocab_size),dtype='float32')
    for t, char in enumerate(input_text):
        input_seq[0, t, in_token_index[char]] = 1.
    decoded_sentence = decode_seq(input_seq)
    print('Bot Reply (Decoded sentence):', decoded_sentence)

In [68]:
response("제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.")

Bot Reply (Decoded sentence):                                                                                
